# 08 - RandomForest

Realizando a predição com DecisionTree. Lembrando que a acurácia para os dados de treino precisa ser superior a 78% para que seja considerada válida para testarmos com os dados de teste e que, nos dados de teste, precisa ser superior a **76.5%** para ser considerada superior a `DecisionTree`.

## Preparando o ambiente

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, GroupKFold, cross_validate
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, Normalizer, RobustScaler

In [5]:
SEED = 5
np.random.seed(SEED)

## Carregando os dados

In [3]:
titanic = pd.read_csv('https://raw.githubusercontent.com/SalatielBairros/kaggle-titanic/main/data/processed/train_processed_byage_ag_nm.csv')
titanic.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Floor,Relateds,faixa_etaria,acompanhado,possui_cabine
0,0,3,0,22,1,0,0,1,1,1,0
1,0,3,0,22,0,0,0,0,1,0,0
2,0,3,0,22,0,0,0,0,1,0,0
3,0,3,0,22,0,0,0,0,1,0,0
4,0,3,0,22,0,0,0,0,1,0,0


In [11]:
teste = pd.read_csv('https://raw.githubusercontent.com/SalatielBairros/kaggle-titanic/main/data/processed/test_processed_byage_ag_nm.csv')
Xvalidacao = teste.drop(columns=['PassengerId'])
Xvalidacao.head(5)

,Pclass,Sex,Age,SibSp,Parch,Floor,Relateds,possui_cabine,acompanhado,faixa_etaria
0,3,0,34,0,0,0,0,0,0,1
1,3,1,47,1,0,0,1,0,1,2
2,2,0,62,0,0,0,0,0,0,2
3,3,0,27,0,0,0,0,0,0,1
4,3,1,22,1,1,0,2,0,1,1


### Separando treino e teste

In [6]:
X = titanic.drop(columns=['Survived'])
y = titanic['Survived']
treino_x, teste_x, treino_y, teste_y = train_test_split(X, y, test_size = 0.2, stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 712 elementos e testaremos com 179 elementos


## Preparando a `RandomForest`

Iniciamos com uma execução mais "bruta" das árvores, utilizando um `gridsearch` manual.

In [7]:
def rf_grid_search(treino_x, treino_y, teste_x, teste_y):
    n_estimators = [50, 100, 300, 500]
    criterion = ["gini", "entropy"]
    max_depth = [2, 3, 5, 7, 9, 11, 13]
    min_samples_split = [2, 8, 16, 32]
    min_samples_leaf = [1, 2, 8, 16, 32]
    resultados = []

    for c in criterion:
        for md in max_depth:
            for ms in min_samples_split:
                for ml in min_samples_leaf:
                    for ne in n_estimators:
                        modelo = RandomForestClassifier(n_estimators = ne, criterion = c, max_depth = md, min_samples_split = ms, min_samples_leaf = ml)
                        modelo.fit(treino_x, treino_y)
                        score = modelo.score(teste_x, teste_y)
                        resultados.append({
                            "criterion": c,
                            "max_depth" : md,
                            "min_samples_split": ms,
                            "min_samples_leaf": ml,
                            "n_estimators": ne,
                            "score": score
                        })

    return pd.DataFrame(resultados)


In [8]:
resultados = rf_grid_search(treino_x, treino_y, teste_x, teste_y).sort_values(by='score', ascending=False)
resultados

,criterion,max_depth,min_samples_split,min_samples_leaf,n_estimators,score
80,gini,3,2,1,50,0.849162
108,gini,3,8,8,50,0.843575
100,gini,3,8,1,50,0.843575
228,gini,5,32,8,50,0.837989
487,gini,13,2,2,500,0.837989
...,...,...,...,...,...,...
79,gini,2,32,32,500,0.759777
599,entropy,2,8,32,500,0.759777
638,entropy,2,32,32,300,0.759777
596,entropy,2,8,32,50,0.759777


In [10]:
best_params = resultados.sort_values(by=['score'], ascending=[False]).head(1).drop(columns=['score']).to_dict(orient='records')[0]
best_params

{'criterion': 'gini',
 'max_depth': 3,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'n_estimators': 50}

Treinando a `RandomForest` com os do split para treino

In [12]:
melhor_modelo = RandomForestClassifier().set_params(**best_params)
melhor_modelo.fit(treino_x, treino_y)
predicoes = melhor_modelo.predict(Xvalidacao)

resultado = pd.DataFrame()
resultado['PassengerId'] = teste['PassengerId']
resultado['Survived'] = predicoes
resultado.to_csv('../../data/submissions/random_forest.csv', index=False)

Resultado no Kaggle de apenas 74.6%. E se eu treinar com o dataset inteiro?

In [14]:
melhor_modelo = RandomForestClassifier().set_params(**best_params)
melhor_modelo.fit(X, y)
predicoes = melhor_modelo.predict(Xvalidacao)

resultado = pd.DataFrame()
resultado['PassengerId'] = teste['PassengerId']
resultado['Survived'] = predicoes
resultado.to_csv('../../data/submissions/random_forest.csv', index=False)

Score de **77.5%**, já maior do que o score da `DecisionTree`, mas ainda um score muito pequeno. Parece estar apresentando sinais de problema no tratamento dos dados.

### Testando uma floresta maior do que o que foi parametrizado para comparação:

In [15]:
modelo = RandomForestClassifier(criterion='gini',
                                           n_estimators=1750,
                                           max_depth=7,
                                           min_samples_split=6,
                                           min_samples_leaf=6,
                                           max_features='auto',
                                           oob_score=True,
                                           random_state=SEED,
                                           n_jobs=-1,
                                           verbose=1) 
melhor_modelo.fit(X, y)
predicoes = melhor_modelo.predict(Xvalidacao)
resultado = pd.DataFrame()
resultado['PassengerId'] = teste['PassengerId']
resultado['Survived'] = predicoes
resultado.to_csv('../../data/submissions/random_forest.csv', index=False)

Não apresentou melhora no score para esta árvore.